In [14]:
# Day 5 - Part2 challenge for AOC2019
# https://adventofcode.com/2019/day/5

In [15]:
# this object is where the Intcode program lives and some useful methods
# to fill it, clear it, read from it, write to it
class Memory:
    global set_trace
    def __init__(self, size):
        self.size = size  # must specify the memory size
        self.storage = [None] * size  # initialize with None
     
    # dump out the contents between 2 positions (defaults all the memory)
    def dump(self,start=0,end=-1):
        if(end == -1):
            end = self.size
        for i in range(start, end):
            print("{}: {}".format(i,self.storage[i]))

    # fill the memory with a sequence, starting at a certain position - which defaults to 0
    def fill(self, codelist, start_pos = 0):
        i = start_pos
        for j in codelist:
            self.storage[i] = j
            i+=1
    
    # fill (write to) memory with a sequence, allowing direct or reference starting location
    def mode_fill(self, s, mem_location, mode):
        # mem_location: where to begin fill or to reference to begin
        # mode (0 - position, 1 - immediate)
        if(set_trace): import pdb; pdb.set_trace()
        if(mode == '0'):
            i = self.storage[mem_location]
            for j in s:
                self.storage[i] = j
                i+=1
        elif(mode == '1'):
            i = mem_location
            for j in s:
                self.storage[i] = j
                i+=1

    # read the contents from just one position
    def read(self, pos):
        return self.storage[pos]
   
    # return the value based on parameter mode (position or immediate)
    def mode_read(self, pos, mode):
        if(set_trace): import pdb; pdb.set_trace()
        # mode: 0 - position, 1 - immediate
        if(mode == '0'):
            return int(self.storage[self.storage[pos]])
        elif(mode == '1'):
            return int(self.storage[pos])

    # clear the memory
    def clear(self):
        self.storage = [None] * self.size

In [16]:
# this function will execute a single sequence (operator + subsequent operands and output location)
def exec_seq(ctr, m):
    global set_trace
    # ctr - where to start
    # m - the memory object to work from
    # the first in the sequence is the 'enhnanced opcode' (includes 3 parameter modes in most significant position)
    # zero-pad to 5 - most significant 0 is implied
    enhanced_opcode = str(m.read(ctr)).zfill(5)
    # first 3 characters are the parameter modes
    pmodes = enhanced_opcode[:3]
    # last 2 characters are the opcode
    opcode = enhanced_opcode[3:]
    # if it is 99, we're done
    if(opcode == '99'):
        return("STOP")
    # this is the "add" operator
    elif(opcode == "01"):
        if(set_trace): import pdb; pdb.set_trace()
        operand1 = m.mode_read(ctr+1, pmodes[2])
        operand2 = m.mode_read(ctr+2, pmodes[1])
        if(set_trace): import pdb; pdb.set_trace()
        calcval = operand1 + operand2
    # this is the "multiply" operator
    elif(opcode == "02"):
        if(set_trace): import pdb; pdb.set_trace()
        operand1 = m.mode_read(ctr+1, pmodes[2])
        operand2 = m.mode_read(ctr+2, pmodes[1])
        calcval = operand1 * operand2
    # this is the 'store from input' operator
    elif(opcode == "03"):
        if(set_trace): import pdb; pdb.set_trace()
        # get the input
        val = getfrominput()
        # store it in address provided by next code
        m.mode_fill(val, ctr+1, pmodes[2])
        # return next prog counter
        return(ctr+2)
    # this is the 'send to output' operator
    elif(opcode == "04"):
        if(set_trace): import pdb; pdb.set_trace()
        # read the val (based on mode) specified from next code and send to output
        sendtooutput(m.mode_read(ctr+1, pmodes[2]))
        return(ctr+2)
    # this is 'jump if true'
    elif(opcode == "05"):
        if(m.mode_read(ctr+1, pmodes[2])):
            return(m.mode_read(ctr+2, pmodes[1]))
        else:
            retuurn(ctr+3)
    # this is 'jump if false'
    elif(opcode == "06"):
        if(m.mode_read(ctr+1, pmodes[2])):
            return(ctr+3)
        else:
            retuurn(m.mode_read(ctr+2, pmodes[1]))
    # this is 'less than' operator
    elif(opcode == "7"):
        operand1 = m.mode_read(ctr+1, pmodes[2])
        operand2 = m.mode_read(ctr+2, pmodes[1])
        if(operand1 < operand2):
            storeval = 1
        else:
            storeval = 0
        m.mode_fill([storeval], ctr+3, pmodes[0])
        retrn(ctr+4)
    # this is the 'eqals to' operator
    elif(opcode == "8"):
        operand1 = m.mode_read(ctr+1, pmodes[2])
        operand2 = m.mode_read(ctr+2, pmodes[1])
        if(operand1 == operand2):
            storeval = 1
        else:
            storeval = 0
        m.mode_fill([storeval], ctr+3, pmodes[0])
        return(ctr+4)        
    # if here, then the opcode is not known
    else:
        return("BADOPCODE")
    # fill the output destination with value depending on mode
    if(set_trace): import pdb; pdb.set_trace()
    m.mode_fill([calcval], ctr+3, pmodes[0])
    # and return the next postion in the memory
    return(ctr+4)

In [17]:
def sendtooutput(s):
    print("OUTPUT: {}".format(s))

In [18]:
def getfrominput():
    global inputstring
    return(inputstring)

In [19]:
# this will turn on debugging and set breakpoints
set_trace = False
# the input provided:
inputstring = "1"

# get the full programming sequence (the program)
seq = [3,9,8,9,10,9,4,9,99,-1,8]
# create a computer object with twice the memory of that sequence
mycomp = Memory(len(seq) * 2)
# load the program
mycomp.fill(seq)


In [20]:
# execute the program
ctr = 0
ret = exec_seq(ctr, mycomp)
while(ret != "STOP" and ret != "BADOPCODE"):
    ret = exec_seq(ret, mycomp)
print("FINAL STATE: {}".format(ret))

FINAL STATE: BADOPCODE
